In [11]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import models,transforms
import matplotlib.pyplot as plt
import pickle
from collections import OrderedDict
import csv
import collections
from  PIL import Image
from tqdm.notebook import tqdm_notebook
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')
import math
device = torch.device("mps" if torch.has_mps else "cpu")
print(device)

mps


In [12]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return torch.matmul(A_mA, B_mB.T) / torch.sqrt(torch.matmul(ssA[:, None],ssB[None]))
def cos_sim(a, b, eps=1e-8):
    """
    added eps for numerical stability
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt


In [13]:
reference_outputs = torch.load("outputs/RFW/ft/reference2_outputs.pt")
reference_identities = np.load('outputs/RFW/ft/reference2_identities.npy',allow_pickle=True)
reference_ethnicities = np.load('outputs/RFW/ft/reference2_ethnicities.npy',allow_pickle=True)
reference_faceIDs = np.load('outputs/RFW/ft/reference2_faceIDs.npy',allow_pickle=True)
print(reference_outputs.shape,reference_identities.shape,reference_ethnicities.shape,reference_faceIDs.shape)

candidate_outputs = torch.load("outputs/RFW/ft/candidate2_outputs.pt")
candidate_identities = np.load('outputs/RFW/ft/candidate2_identities.npy',allow_pickle=True)
candidate_ethnicities = np.load('outputs/RFW/ft/candidate2_ethnicities.npy',allow_pickle=True)
candidate_faceIDs = np.load('outputs/RFW/ft/candidate2_faceIDs.npy',allow_pickle=True)
print(candidate_outputs.shape,candidate_identities.shape,candidate_ethnicities.shape,candidate_faceIDs.shape)

candidate_identities

torch.Size([11403, 2048]) (11403,) (11403,) (11403,)
torch.Size([29117, 2048]) (29117,) (29117,) (29117,)


array(['m.0c7mh2', 'm.0c7mh2', 'm.026tq86', ..., 'm.027nbyf', 'm.027nbyf',
       'm.098d5s'], dtype='<U10')

In [14]:
reference_outputs = reference_outputs.to(device)
candidate_outputs = candidate_outputs.to(device)

In [15]:
cor = corr2_coeff(reference_outputs,candidate_outputs).cpu().detach().numpy()
cos = cos_sim(reference_outputs,candidate_outputs).cpu().detach().numpy()

In [16]:
def verification(similarity_mat, thresh,reference_identities,reference_ethnicities,candidate_identities):
    verification = pd.DataFrame(columns=['reference_identity','reference_ethnicity','TPR','TNR','FPR','FNR'])
    match_mat = np.abs(similarity_mat)>=thresh
    for i, match_row in tqdm_notebook(enumerate(match_mat),total=len(match_mat)):
        identity = reference_identities[i]
        ethnicity = reference_ethnicities[i]
        matches = candidate_identities[match_row]  
        not_matches = candidate_identities[~match_row]
        TP = np.sum(matches == identity)
        FP = np.sum(matches != identity)
        TN = np.sum(not_matches != identity)
        FN = np.sum(not_matches == identity)
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        FPR = FP/(TN+FP)
        FNR = FN/(TP+FN)
        row = {'reference_identity': identity,
           'reference_ethnicity': ethnicity, 
           'TPR': TPR,
           'TNR': TNR,
           'FPR': FPR,
           'FNR': FNR}
        verification = verification.append(row,ignore_index=True)
    return verification


In [ ]:
for thresh in np.arange(0.2,0.8,0.1):
    cos_verificaiton = verification(cos, thresh,reference_identities,reference_ethnicities,candidate_identities)
    African_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'African']
    African = np.round(1- African_cos.mean(axis=0)['FNR'],4)
    AfricanFPR = np.round(African_cos.mean(axis=0)['FPR'],4)

    Asian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Asian']
    Asian = np.round(1- Asian_cos.mean(axis=0)['FNR'],4)
    AsianFPR = np.round(Asian_cos.mean(axis=0)['FPR'],4)

    Indian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Indian']
    Indian = np.round(1- Indian_cos.mean(axis=0)['FNR'],4)
    IndianFPR = np.round(Indian_cos.mean(axis=0)['FPR'],4)

    Caucasian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Caucasian']
    Caucasian = np.round(1- Caucasian_cos.mean(axis=0)['FNR'],4)
    CaucasianFPR = np.round(Caucasian_cos.mean(axis=0)['FPR'],4)

    print('GAR for threshold =', np.round(thresh,2),', African:', African,'Asian:', Asian,'Indian:', Indian,'Caucasian:', Caucasian)
    print('FAR for threshold =', np.round(thresh,2),', African:', AfricanFPR,'Asian:', AsianFPR,'Indian:', IndianFPR,'Caucasian:', CaucasianFPR)



In [ ]:
for thresh in np.arange(0.2,0.6,0.05):
    cos_verificaiton = verification(cor, thresh,reference_identities,reference_ethnicities,candidate_identities)
    African_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'African']
    African = np.round(1- African_cos.mean(axis=0)['FNR'],4)
    AfricanFPR = np.round(African_cos.mean(axis=0)['FPR'],4)

    Asian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Asian']
    Asian = np.round(1- Asian_cos.mean(axis=0)['FNR'],4)
    AsianFPR = np.round(Asian_cos.mean(axis=0)['FPR'],4)

    Indian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Indian']
    Indian = np.round(1- Indian_cos.mean(axis=0)['FNR'],4)
    IndianFPR = np.round(Indian_cos.mean(axis=0)['FPR'],4)

    Caucasian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Caucasian']
    Caucasian = np.round(1- Caucasian_cos.mean(axis=0)['FNR'],4)
    CaucasianFPR = np.round(Caucasian_cos.mean(axis=0)['FPR'],4)

    print('GAR for threshold =', np.round(thresh,2),', African:', African,'Asian:', Asian,'Indian:', Indian,'Caucasian:', Caucasian)
    print('FAR for threshold =', np.round(thresh,2),', African:', AfricanFPR,'Asian:', AsianFPR,'Indian:', IndianFPR,'Caucasian:', CaucasianFPR)



In [54]:
for thresh in np.arange(0.393,0.46,0.0005):
    cos_verificaiton = verification(cos, thresh,reference_identities,reference_ethnicities,candidate_identities)
    Caucasian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Caucasian']
    print('thresh = ', thresh, ', FPR = ', np.round(Caucasian_cos.mean(axis=0)['FPR'],4), ", GAR = ",  np.round(1- Caucasian_cos.mean(axis=0)['FNR'],4))

  0%|          | 0/11403 [00:00<?, ?it/s]

thresh =  0.393 , FPR =  0.0105 , GAR =  0.9404


  0%|          | 0/11403 [00:00<?, ?it/s]

thresh =  0.3935 , FPR =  0.0103 , GAR =  0.9401


  0%|          | 0/11403 [00:00<?, ?it/s]

thresh =  0.394 , FPR =  0.0102 , GAR =  0.9399


  0%|          | 0/11403 [00:00<?, ?it/s]

thresh =  0.3945 , FPR =  0.01 , GAR =  0.9387


  0%|          | 0/11403 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for thresh in np.arange(0.265,0.275,0.0025):
    cor_verificaiton = verification(cor, thresh,reference_identities,reference_ethnicities,candidate_identities)
    print('thresh = ', thresh, 'FPR = ', np.round(cor_verificaiton.mean(axis=0)['FPR'],4))

In [56]:
african_cos_thresh = 0.4515
asian_cos_thresh = 0.4572
indian_cos_thresh = 0.441
caucasian_cos_thresh = 0.3945

In [58]:
cos_verificaiton = verification(cos, 0,reference_identities,reference_ethnicities,candidate_identities)
African_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'African']
African = np.round(1- African_cos.mean(axis=0)['FNR'],4)
AfricanFPR = np.round(African_cos.mean(axis=0)['FPR'],4)

Asian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Asian']
Asian = np.round(1- Asian_cos.mean(axis=0)['FNR'],4)
AsianFPR = np.round(Asian_cos.mean(axis=0)['FPR'],4)

Indian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Indian']
Indian = np.round(1- Indian_cos.mean(axis=0)['FNR'],4)
IndianFPR = np.round(Indian_cos.mean(axis=0)['FPR'],4)

Caucasian_cos = cos_verificaiton.loc[cos_verificaiton.reference_ethnicity == 'Caucasian']
Caucasian = np.round(1- Caucasian_cos.mean(axis=0)['FNR'],4)
CaucasianFPR = np.round(Caucasian_cos.mean(axis=0)['FPR'],4)

print('GAR for threshold =', np.round(thresh,2),', African:', African,'Asian:', Asian,'Indian:', Indian,'Caucasian:', Caucasian)
print('FAR for threshold =', np.round(thresh,2),', African:', AfricanFPR,'Asian:', AsianFPR,'Indian:', IndianFPR,'Caucasian:', CaucasianFPR)



  0%|          | 0/11403 [00:00<?, ?it/s]

GAR for threshold = 0.4 , African: 1.0 Asian: 1.0 Indian: 1.0 Caucasian: 1.0
FAR for threshold = 0.4 , African: 1.0 Asian: 1.0 Indian: 1.0 Caucasian: 1.0


In [13]:
cor_verificaiton = verification(cor, cor_thresh,reference_identities,reference_ethnicities,candidate_identities)
African_cor = cor_verificaiton.loc[cor_verificaiton.reference_ethnicity == 'African']
African = np.round(1- African_cor.mean(axis=0)['FNR'],4)
AfricanFPR = np.round(African_cor.mean(axis=0)['FPR'],4)

Asian_cor = cor_verificaiton.loc[cor_verificaiton.reference_ethnicity == 'Asian']
Asian = np.round(1- Asian_cor.mean(axis=0)['FNR'],4)
AsianFPR = np.round(Asian_cor.mean(axis=0)['FPR'],4)

Indian_cor = cor_verificaiton.loc[cor_verificaiton.reference_ethnicity == 'Indian']
Indian = np.round(1- Indian_cor.mean(axis=0)['FNR'],4)
IndianFPR = np.round(Indian_cor.mean(axis=0)['FPR'],4)

Caucasian_cor = cor_verificaiton.loc[cor_verificaiton.reference_ethnicity == 'Caucasian']
Caucasian = np.round(1- Caucasian_cor.mean(axis=0)['FNR'],4)
CaucasianFPR = np.round(Caucasian_cor.mean(axis=0)['FPR'],4)

print('GAR for threshold =', np.round(thresh,2),', African:', African,'Asian:', Asian,'Indian:', Indian,'Caucasian:', Caucasian)
print('FAR for threshold =', np.round(thresh,2),', African:', AfricanFPR,'Asian:', AsianFPR,'Indian:', IndianFPR,'Caucasian:', CaucasianFPR)



  0%|          | 0/11403 [00:00<?, ?it/s]

GAR for threshold = 0.27 , African: 0.9373 Asian: 0.9374 Indian: 0.9328 Caucasian: 0.872
FAR for threshold = 0.27 , African: 0.0127 Asian: 0.0165 Indian: 0.0107 Caucasian: 0.0025
